In [2]:
import numpy as np
import csv
import openpyxl
workbook_filename = 'case39.xlsx'
wb = openpyxl.load_workbook(workbook_filename)
#get NB and NPV
ws = wb['bus']
i=2
max=0
NPV=0
while(ws.cell(row=i,column=1).value != None):
    if ws.cell(row=i,column=1).value > max:
        max = ws.cell(row=i,column=1).value
    if ws.cell(row=i,column=2).value == 2:
        NPV += 1
    i += 1
NB = max

# Load parameters
NX = 2 * NB - NPV - 2
BMVA = 100.0
EPS = 1.0e-6
RTOD = 180.0 / np.pi

# Load charging VA between buses
ws = wb['line']
CHVA = np.zeros((NB, NB), dtype=complex)
i=2
while(ws.cell(row=i,column=1).value != None):
    
    index1 = ws.cell(row=i,column=1).value - 1 #bus1 - 1
    index2 = ws.cell(row=i,column=2).value - 1 #bus2 - 1
    CHVA[index1, index2] = ws.cell(row=i,column=5).value
    i += 1
# print(CHVA)

# Load line impedances
ZL = np.zeros((NB, NB), dtype=complex)
tap = np.zeros((NB, NB), dtype=float)
i=2
while(ws.cell(row=i,column=1).value != None):
    index1 = ws.cell(row=i,column=1).value - 1 #bus1 - 1
    index2 = ws.cell(row=i,column=2).value - 1 #bus2 - 1
    R = ws.cell(row=i,column=3).value
    X = ws.cell(row=i,column=4).value
    ZL[index1,index2] = 1/complex(R,X)
    tap[index1,index2]= ws.cell(row=i,column=7).value
    i += 1
# print(ZL)
# print(tap)


# Bus type: 1 for swing bus, 2 for PV bus, 3 for PQ bus(Load bus)
ws = wb['bus']
i=2
BT = np.zeros((NB))
while(ws.cell(row=i,column=1).value != None):
    idxBus = ws.cell(row=i,column=1).value - 1 #bus number - 1
    BT[idxBus] = ws.cell(row=i,column=2).value
    i += 1
# print(BT)

# Bus data
i=2
PG = np.zeros((NB))
QG = np.zeros((NB))
PL = np.zeros((NB))
QL = np.zeros((NB))
V = np.zeros((NB))
ANG = np.zeros((NB))
while(ws.cell(row=i,column=1).value != None):
    idxBus = ws.cell(row=i,column=1).value - 1 #bus number - 1
    PG[idxBus] = ws.cell(row=i,column=5).value
    QG[idxBus] = ws.cell(row=i,column=6).value
    PL[idxBus] = ws.cell(row=i,column=7).value
    QL[idxBus] = ws.cell(row=i,column=8).value
    V[idxBus] = ws.cell(row=i,column=3).value
    ANG[idxBus] = ws.cell(row=i,column=4).value
    i += 1
#print(PG)
#print(QG)
#print(PL)
#print(QL)
#print(V)
#print(ANG)

# Formulate Ybus
i = 1j
YB = np.zeros((NB, NB), dtype=complex)
for NI in range(NB):
    for NJ in range(NB):
        if tap[NI,NJ] == 0:
            YB[NI,NJ]= YB[NI,NJ] - ZL[NI,NJ]
        else :
            YB[NI,NJ]= YB[NI,NJ] - ZL[NI,NJ]/tap[NI,NJ]
        YB[NJ,NI]= YB[NI,NJ]
ws = wb['line']
ws1 = wb['bus']
m=ws.max_row
k=ws1.max_row
for l in range(2,m,1):
    for j in range(2,k,1):
        if ws.cell(row=l,column=1).value == ws1.cell(row=j,column=1).value:
            index1 = ws.cell(row=l,column=1).value - 1 #bus1 - 1
            index2 = ws.cell(row=l,column=2).value - 1 #bus2 - 1            
            A=tap[index1,index2]
            A=A*A            
            YB[index1,index1]= YB[index1,index1] + (ZL[index1,index2]/A) + i*CHVA[index1,index2]
        elif ws.cell(row=l,column=2).value == ws1.cell(row=j,column=1).value:
            j=j-2
            index1 = ws.cell(row=l,column=1).value - 1 #bus1 - 1
            index2 = ws.cell(row=l,column=2).value - 1 #bus2 - 1 
            YB[index2,index2]= YB[index2,index2] +ZL[index1,index2] + i*CHVA[index1,index2]
# Prepare X vector
NA = 0
NV = 0
NXA = []
NXV = []

for NI in range(NB):
    if BT[NI] == 2:
        NA = NA+1
        NXA.append(NI)
    if BT[NI] == 3:
        NA = NA+1
        NXA.append(NI)
        NV = NV+1
        NXV.append(NI)

# Save X with initial values
XX = np.zeros(NA + NV, dtype=float)

for NI in range(NA):
    XX[NI] = ANG[NXA[NI]]

for NI in range(NV):
    XX[NA + NI] = V[NXV[NI]]

NITER = 0

# Newton-Raphson's Loop
FLAG = 1

while FLAG == 1:
    for NI in range(NA):
        ANG[NXA[NI]] = XX[NI]

    for NI in range(NV):
        V[NXV[NI]] = XX[NA + NI]

    # Find bus P and Q
    P = np.zeros(NB)
    Q = np.zeros(NB)

    for NI in range(NB):
        P[NI]=0.0
        Q[NI]=0.0
        for NJ in range(NB):
            P[NI] += V[NI] * V[NJ] * (np.real(YB[NI, NJ]) * np.cos((ANG[NI] - ANG[NJ]) / RTOD)
                                      + np.imag(YB[NI, NJ]) * np.sin((ANG[NI] - ANG[NJ]) / RTOD))

            Q[NI] += V[NI] * V[NJ] * (np.real(YB[NI, NJ]) * np.sin((ANG[NI] - ANG[NJ]) / RTOD)
                                      - np.imag(YB[NI, NJ]) * np.cos((ANG[NI] - ANG[NJ]) / RTOD))

    # Check errors of buses' P and Q
    FX = np.zeros(NA + NV)

    for NI in range(NA):
        FX[NI] = (PG[NXA[NI]] - PL[NXA[NI]]) / BMVA - P[NXA[NI]]

    for NI in range(NV):
        FX[NA + NI] = (QG[NXV[NI]] - QL[NXV[NI]]) / BMVA - Q[NXV[NI]]

    NI = 0
    FLAG = 0

    while NI < NX:
        if np.abs(FX[NI]) > EPS:
            FLAG = 1
            NI = NX
        NI += 1

    # Jacobian Matrix Creation
    if FLAG == 1:
        J = np.zeros((NA + NV, NA + NV))

        # J11
        for NI in range(NA):
            for NJ in range(NA):
                NII = NXA[NI]
                NJJ = NXA[NJ]

                if NII != NJJ:
                    J[NI, NJ] = V[NII] * V[NJJ] * (
                            np.real(YB[NII, NJJ]) * np.sin((ANG[NII] - ANG[NJJ]) / RTOD)
                            - np.imag(YB[NII, NJJ]) * np.cos((ANG[NII] - ANG[NJJ]) / RTOD))
                else:
                    J[NI, NJ] = 0.0
                    for NK in range(NB):
                        if NK != NII:
                            J[NI, NJ] += V[NII] * V[NK] * (-np.real(YB[NII, NK]) * np.sin((ANG[NII] - ANG[NK]) / RTOD)
                                                           + np.imag(YB[NII, NK]) * np.cos((ANG[NII] - ANG[NK]) / RTOD))

        # J12
        for NI in range(NA):
            for NJ in range(NV):
                NII = NXA[NI]
                NJJ = NXV[NJ]

                if NII == NJJ:
                    J[NI, NA + NJ] = 0.0
                    for NK in range(NB):
                        if NII != NK:
                            J[NI, NA + NJ] += V[NK] * (np.real(YB[NII, NK]) * np.cos((ANG[NII] - ANG[NK]) / RTOD)
                                                       + np.imag(YB[NII, NK]) * np.sin((ANG[NII] - ANG[NK]) / RTOD))
                    J[NI, NA + NJ] += 2.0 * V[NII] * np.real(YB[NII, NII])
                else:
                    J[NI, NA + NJ] = V[NII] * (np.real(YB[NII, NJJ]) * np.cos((ANG[NII] - ANG[NJJ]) / RTOD)
                                               + np.imag(YB[NII, NJJ]) * np.sin((ANG[NII] - ANG[NJJ]) / RTOD))

        # J21
        for NI in range(NV):
            for NJ in range(NA):
                NII = NXV[NI]
                NJJ = NXA[NJ]

                if NII == NJJ:
                    J[NA + NI, NJ] = 0.0
                    for NK in range(NB):
                        if NII != NK:
                            J[NA + NI, NJ] += V[NII] * V[NK] * (
                                    np.real(YB[NII, NK]) * np.cos((ANG[NII] - ANG[NK]) / RTOD)
                                    + np.imag(YB[NII, NK]) * np.sin((ANG[NII] - ANG[NK]) / RTOD))
                else:
                    J[NA + NI, NJ] = -V[NII] * V[NJJ] * (
                            np.real(YB[NII, NJJ]) * np.cos((ANG[NII] - ANG[NJJ]) / RTOD)
                            + np.imag(YB[NII, NJJ]) * np.sin((ANG[NII] - ANG[NJJ]) / RTOD))

        # J22
        for NI in range(NV):
            for NJ in range(NV):
                NII = NXV[NI]
                NJJ = NXV[NJ]

                if NII != NJJ:
                    J[NA + NI, NA + NJ] = V[NJJ] * (
                            np.real(YB[NII, NJJ]) * np.sin((ANG[NII] - ANG[NJJ]) / RTOD)
                            - np.imag(YB[NII, NJJ]) * np.cos((ANG[NII] - ANG[NJJ]) / RTOD))
                else:
                    J[NA + NI, NA + NJ] = 0.0
                    for NK in range(NB):
                        if NK != NII:
                            J[NA + NI, NA + NJ] += V[NK] * (
                                    np.real(YB[NII, NK]) * np.sin((ANG[NII] - ANG[NK]) / RTOD)
                                    - np.imag(YB[NII, NK]) * np.cos((ANG[NII] - ANG[NK]) / RTOD))
                    J[NA + NI, NA + NJ] -= 2.0 * V[NII] * np.imag(YB[NII, NII])
        # with open('57J_matrix.csv', 'w', newline='') as csvfile:
        #     writer = csv.writer(csvfile)
        #     writer.writerows(J)
        INVJ = np.linalg.inv(J)
        DX= np.zeros((NX), dtype=float)
        for NI in range(NX):
            DX[NI]=0.0
            for NJ in range(NX):
                DX[NI] += INVJ[NI,NJ]*FX[NJ]
        
        for NI in range(NX):
            XX[NI] = XX[NI]+DX[NI]

        NITER += 1
print(NITER)
for NI in range(NB):
    if BT[NI] == 1:
        PG[NI] = P[NI] * BMVA + PL[NI]
        QG[NI] = Q[NI] * BMVA + QL[NI]
    if BT[NI] == 2:
        QG[NI] = Q[NI] * BMVA + QL[NI]

# Display results and save to CSV
header = ["Bus", "Type", "Voltage", "Angle", "PG", "QG", "PL", "QL"]
data = []

for NI in range(NB):
    data.append([NI + 1, BT[NI], V[NI], ANG[NI], PG[NI], QG[NI], PL[NI], QL[NI]])

# Print to console
print(" BUS NAME TYPE  VOLTAGE ANGLE PG  QG  PL QL")
for row in data:
    print(row)

# Save to CSV
with open(f"{workbook_filename} result.csv", mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(data)

906
 BUS NAME TYPE  VOLTAGE ANGLE PG  QG  PL QL
[1, 3.0, 1.0448420176688908, -8.813631380353076, 0.0, 0.0, 0.0, 0.0]
[2, 3.0, 1.0420448714567716, -6.1597847782104695, 0.0, 0.0, 0.0, 0.0]
[3, 3.0, 1.0139920571837466, -9.020060374637433, 0.0, 0.0, 322.0, 2.4]
[4, 3.0, 0.9698211541586179, -9.787352045284415, 0.0, 100.0, 500.0, 184.0]
[5, 3.0, 0.9475847441347474, -8.410473524529044, 0.0, 200.0, 0.0, 0.0]
[6, 3.0, 0.9398137655173175, -7.565082139703752, 0.0, 0.0, 0.0, 0.0]
[7, 3.0, 0.9355162282745186, -10.095096523142837, 0.0, 0.0, 233.8, 84.0]
[8, 3.0, 0.9381418685084505, -10.679152495353586, 0.0, 0.0, 522.0, 176.0]
[9, 3.0, 1.0043403658507326, -10.566090094505798, 0.0, 0.0, 0.0, 0.0]
[10, 3.0, 0.9767378178119731, -5.088554575873221, 0.0, 0.0, 0.0, 0.0]
[11, 3.0, 0.9634781511203662, -5.917920522280584, 0.0, 0.0, 0.0, 0.0]
[12, 3.0, 0.948962462619017, -5.958470641488476, 0.0, 0.0, 7.5, 88.0]
[13, 3.0, 0.975189857726323, -5.883026507043947, 0.0, 0.0, 0.0, 0.0]
[14, 3.0, 0.978641715971029, -7